In [1]:
import sys
sys.path.append('/Users/Tim/PycharmProjects/HOI/')

In [2]:
import sys
sys.path

['/Users/tim/PycharmProjects/HOI/examples',
 '/Users/tim/opt/anaconda3/envs/hoi/lib/python310.zip',
 '/Users/tim/opt/anaconda3/envs/hoi/lib/python3.10',
 '/Users/tim/opt/anaconda3/envs/hoi/lib/python3.10/lib-dynload',
 '',
 '/Users/tim/opt/anaconda3/envs/hoi/lib/python3.10/site-packages',
 '/Users/Tim/PycharmProjects/HOI/']

In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [4]:
from synthetic_data import stationary_pb_ts
from HOI.preprocessings import compute_kernel
from HOI.tests import test_independence, estimate_tail_head, shifting_kernel, permutation_stationary_ts
from HOI.statistics import compute_dHSIC_statistics

In [5]:
import statsmodels.api as sm

In [29]:
df = stationary_pb_ts(n_sample = 300, seed = 11, d = 0.5, mode = "case1", a=0.5)

In [30]:
df

,d1,d2,d3
0,1.749455,-0.286073,-0.484565
1,-1.778591,-0.151321,-0.619096
2,-0.573893,0.345390,-1.728589
3,-0.762680,0.862377,-2.450439
4,-1.500815,1.168026,-2.043612
...,...,...,...
295,0.715899,0.676237,1.623845
296,0.733380,0.567582,0.646624
297,-2.280090,-0.053891,1.082430
298,-1.936439,-2.526821,0.738789


In [31]:
data_dict, kernel_dict = compute_kernel(df)

In [32]:
data_list = [data_dict['d1'], data_dict['d2'], data_dict['d3']]

In [33]:
k_list = [kernel_dict['d1'], kernel_dict['d2'], kernel_dict['d3']]

In [34]:
statistic = compute_dHSIC_statistics(k_list)
critical_value = permutation_stationary_ts(data_list, k_list, alpha=0.05)

In [35]:
test_independence(k_list=k_list, data_list=data_list, mode='stat_ts', n_perms=1000, alpha=0.05)

(0.0026235365848013004, 0.0017333242528891895, 1)

In [36]:
test_independence(k_list=[kernel_dict['d1'], kernel_dict['d2']], 
                  data_list=[data_dict['d1'], data_dict['d2']], 
                  mode='stat_ts', n_perms=1000, alpha=0.05)

(0.0009016972028506043, 0.0020964309276277015, 0)

In [37]:
test_independence(k_list=[kernel_dict['d1'], kernel_dict['d3']], 
                  data_list=[data_dict['d1'], data_dict['d3']], 
                  mode='stat_ts', n_perms=1000, alpha=0.05)

(0.0006952669977832171, 0.001652320236389282, 0)

In [38]:
test_independence(k_list=[kernel_dict['d2'], kernel_dict['d3']], 
                  data_list=[data_dict['d2'], data_dict['d3']], 
                  mode='stat_ts', n_perms=1000, alpha=0.05)

(0.0003036424696591933, 0.001733001524319322, 0)

In [ ]:
def shifting_kernel(head, tail, k_list):
    num_var = len(k_list)
    shifted_kernel_lists = {}
    for cut in range(head, tail):
        ind = list(range(cut, tail)) + list(range(0, cut))
        shifted_kernel_list = {}
        for i in range(num_var):
            shifted_kernel_list[str(i)] = k_list[i][:, ind]
        shifted_kernel_lists[str(cut-head)] = shifted_kernel_list
    return shifted_kernel_lists

In [ ]:
shifted_kernel_lists = shifting_kernel(head = 7, tail = 200, 
                                    k_list = [kernel_dict['d1'], kernel_dict['d2'], kernel_dict['d3']])

In [ ]:
k_list = [kernel_dict['d1'], kernel_dict['d2'], kernel_dict['d3']]

In [ ]:
k_list[0].shape

In [ ]:
def rand_ind(num_cut, num_var):
    perms = []
    perms.append(np.array(range(num_cut)))
    for i in range(num_var-1):    
        perm = np.random.permutation(num_cut)
        perms.append(perm)
    perms = np.transpose(perms)
    return perms

In [ ]:
def permutation_stationary_ts(data_list, k_list, alpha):
    num_var = len(k_list)
    tail, head = estimate_tail_head(data_list)
    shifted_kernel_lists = shifting_kernel(head, tail, k_list)
    num_cut = len(shifted_kernel_lists)
    perms = rand_ind(num_cut, num_var)
    
    shifted_dHSICs = []
    for ind in perms:
        shifted_k_lists = [shifted_kernel_lists[str(ind[i])][str(i)] for i in range(num_var)]
        shifted_dHSIC = compute_dHSIC_statistics(shifted_k_lists)
        shifted_dHSICs.append(shifted_dHSIC)

    critical_value = np.quantile(shifted_dHSICs, 1 - alpha)
    return critical_value

In [ ]:
data_list = [data_dict['d1'], data_dict['d3'], data_dict['d3']]
k_list = [kernel_dict['d1'], kernel_dict['d2'], kernel_dict['d3']]
alpha = 0.05

In [ ]:
critical = permutation_stationary_ts(data_list, k_list, alpha)

In [ ]:
critical

In [ ]:
num_var = len(k_list)
tail, head = estimate_tail_head(data_list)
shifted_kernel_lists = shift_kernel(head, tail, k_list)
num_cut = len(shifted_kernel_lists)
perms = rand_ind(num_cut, num_var)

In [ ]:
head, tail

In [ ]:

shifted_dHSICs = []
for ind in perms:
    shifted_k_lists = [shifted_kernel_lists[str(ind[i])][str(i)] for i in range(num_var)]
    shifted_dHSIC = compute_dHSIC_statistics(shifted_k_lists)
    shifted_dHSICs.append(shifted_dHSIC)

In [ ]:
shifted_dHSICs

In [ ]:
shifted_kernel_lists[str(1)][str(1)].shape